# Веб-скрейпинг и таблицы

*Алла Тамбовцева, НИУ ВШЭ*

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [2]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url, ) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [3]:
page  # response 200 - страница загружена

<Response [200]>

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

In [4]:
from bs4 import BeautifulSoup  # не спрашивайте, почему BeautifulSoup

In [5]:
soup = BeautifulSoup(page.text, 'lxml')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [6]:
soup

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>N + 1 — главное издание о науке, технике и технологиях</title>
<link as="style" href="/front-build/css/main.css" rel="preload"/>
<link as="style" href="/front-build/css/app.css?v=10" rel="preload"/>
<link as="script" href="/front-build/js/app.js?v=10" rel="preload"/>
<link as="font" crossorigin="" href="https://storage.yandexcloud.net/staticn1/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://storage.yandexcloud.net/staticn1/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css" rel="stylesheet"/>
<link href="/front-build/css/app.css?v=10" rel="stylesheet"/>
<link href="https://storage.yandexcloud.net/staticn1/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="h

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   N + 1 — главное издание о науке, технике и технологиях
  </title>
  <link as="style" href="/front-build/css/main.css" rel="preload"/>
  <link as="style" href="/front-build/css/app.css?v=10" rel="preload"/>
  <link as="script" href="/front-build/js/app.js?v=10" rel="preload"/>
  <link as="font" crossorigin="" href="https://storage.yandexcloud.net/staticn1/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://storage.yandexcloud.net/staticn1/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
  <link href="/front-build/css/main.css" rel="stylesheet"/>
  <link href="/front-build/css/app.css?v=10" rel="stylesheet"/>
  <link href="https://storage.yandexcloud.net/staticn1/favicon/apple-touch-icon.png" rel="apple-touch-icon" s

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Посмотрим на кусочки кода, соответствующие всем ссылкам на главной странице сайта:

In [8]:
for link in soup.find_all('a'):
    print(link.get('href'))

/search
https://offline.nplus1.ru/
https://nplus.pro/
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
/search/empty/869
/search/empty/874
/search/empty/880
/search/empty/768
/search/empty/890
/search/empty/871
/search/empty/876
/search/empty/775
/search/empty/767
/search/empty/771
/search/empty/772
/search/empty/778
/search/empty/917
/search/empty/918
/search/empty/824
https://t.me/nplusone
https://vk.com/nplusone
https://ok.ru/nplus1
https://twitter.com/nplusodin
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/news/2015/09/21/editor-thy-name
/search/empty/869
/search/empty/874
/search/empty/880
/search/empty/768
/search/empty/890
/search/empty/871
/search/empty/876
/search/empty/775
/search/empty/767
/search/empty/771
/search/empty/772
/search/empty/778
/search/empty/917
/search/empty/918
/search/empty/824
https://offline.nplus1.ru/
https://nplus.pro/
https://t.me/nplusone
h

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [9]:
urls = []

for link in soup.find_all('a'):
    if '/news' in link.get('href'):
        if 'https://' in link.get('href'):
            urls.append(link.get('href'))
urls

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/03/06/mother-and-child',
 'https://nplus1.ru/news/2023/03/04/vega-c-second-fail',
 'https://nplus1.ru/news/2023/03/04/hubble-and-satellites',
 'https://nplus1.ru/news/2023/03/04/18O-alpha-gamma-22Ne',
 'https://nplus1.ru/news/2023/03/04/10-sec-quantum-memory',
 'https://nplus1.ru/news/2023/03/04/Lane-nucleation',
 'https://nplus1.ru/news/2023/03/03/earliest-horsemanship',
 'https://nplus1.ru/news/2023/03/03/dogs-of-Chernobyl',
 'https://nplus1.ru/news/2023/03/03/ford-in-sixty-seconds',
 'https://nplus1.ru/news/2023/03/03/childhood-maltreatment-and-offspring',
 'https://nplus1.ru/news/2023/03/03/metformin-for-pregnant',
 'https://nplus1.ru/news/2023/03/03/very-slow-tunnelling',
 'https://nplus1.ru/news/2023/03/03/sagging-skin',
 'https://nplus1.ru/news/2023/03/03/p38-mapk-AD',
 'https://nplus1.ru/news/2023/03/03/ryugu-organics',
 'https://nplus1.ru/news/2023/03/03/structure-in-khufus-pyramid',
 'https://

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [30]:
url0 = urls[1]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'lxml')

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [31]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="#f26e40" name="msapplication-TileColor"/>,
 <meta content="#ffffff" name="theme-color"/>,
 <meta content="8c90b02c84ac3b72" name="yandex-verification"/>,
 <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" name="description"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:site_name"/>,
 <meta content="Во Франции нашли останки держащихся за руки мамы и ребенка" property="og:title"/>,
 <meta content="https://app-images.website.yandexcloud.net/761519/640550d796a8f_cover_share.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2023/03/06/mother-and-child" property="og:url"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:description"/>,
 <meta content="article" property="og:type"/>,
 <me

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [32]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Михаил Подрезов" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [33]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Михаил Подрезов" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [35]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Михаил Подрезов'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [37]:
author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
author

'Михаил Подрезов'

Аналогичным образом извлечем дату, заголовок и описание.

In [38]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']

Не прошло и двух пар, как мы разобрались со всем :) Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [39]:
def GetNews(url0):
    """
    Returns a tuple with url0, date, author, description, title, final_text, rubrics, diff.
    Parameters:
    
    url0 is a link to the news (string)
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    
    author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
 
    
    return url0, date, author, title

Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [40]:
from time import sleep

In [41]:
len(urls)

65

In [42]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in urls[:50]:
    res = GetNews(link)
    news.append(res)
    sleep(1) # задержка в 3 секунды

In [43]:
news[0]

('https://nplus1.ru/news/2015/09/21/editor-thy-name',
 '2015-09-21',
 'Андрей Коняев',
 'Мы ищем редакторов: физиков, археологов, медиков')

Так теперь выглядит первый элемент списка:

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [44]:
import pandas as pd

In [45]:
df = pd.DataFrame(news)

In [46]:
df.head(10)

,0,1,2,3
0,https://nplus1.ru/news/2015/09/21/editor-thy-name,2015-09-21,Андрей Коняев,"Мы ищем редакторов: физиков, археологов, медиков"
1,https://nplus1.ru/news/2023/03/06/mother-and-c...,2023-03-06,Михаил Подрезов,Во Франции нашли останки держащихся за руки ма...
2,https://nplus1.ru/news/2023/03/04/vega-c-secon...,2023-03-04,Александр Войтюк,Аварийный пуск ракеты Vega-С связали с разруше...
3,https://nplus1.ru/news/2023/03/04/hubble-and-s...,2023-03-04,Александр Войтюк,Околоземные спутники испортили снимки телескоп...
4,https://nplus1.ru/news/2023/03/04/18O-alpha-ga...,2023-03-04,Марат Хамадеев,Физики из JUNA уточнили параметры превращения ...
5,https://nplus1.ru/news/2023/03/04/10-sec-quant...,2023-03-04,Марат Хамадеев,Память для узла квантовой сети на основе ионов...
6,https://nplus1.ru/news/2023/03/04/Lane-nucleation,2023-03-04,Марат Хамадеев,Теорию разбиения встречных потоков на полосы п...
7,https://nplus1.ru/news/2023/03/03/earliest-hor...,2023-03-03,Михаил Подрезов,Антропологи нашли древнейших всадников. Ими ок...
8,https://nplus1.ru/news/2023/03/03/dogs-of-Cher...,2023-03-03,Слава Гоменюк,Чернобыльских собак объединили в отдельные поп...
9,https://nplus1.ru/news/2023/03/03/ford-in-sixt...,2023-03-03,Николай Воронцов,Ford запатентовал самоугоняемый автомобиль


Переименуем столбцы в базе.

In [47]:
df.columns = ['link', 'date', 'author', 'title']

In [48]:
df.head(2)

,link,date,author,title
0,https://nplus1.ru/news/2015/09/21/editor-thy-name,2015-09-21,Андрей Коняев,"Мы ищем редакторов: физиков, археологов, медиков"
1,https://nplus1.ru/news/2023/03/06/mother-and-c...,2023-03-06,Михаил Подрезов,Во Франции нашли останки держащихся за руки ма...


Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [50]:
df.to_excel('nplus-news.xlsx')